In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

C:\Users\anspa\miniconda3\envs\solaris\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\anspa\miniconda3\envs\solaris\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\anspa\miniconda3\envs\solaris\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\anspa\miniconda3\envs\solaris\lib\site-packages\tensorf

In [2]:
batch_size = 12

## Touple Image - Class Label

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [4]:
x_train

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [22]:
# The dataset expects (BS, H, W, C), so:

img_rows, img_cols, img_ch = 28, 28, 1
input_shape = (img_rows, img_cols, img_ch)
x_train = x_train.reshape(x_train.shape[0], *input_shape)
# OR x_train = np.expand_dims(x_train, -1)
x_test = x_test.reshape(x_test.shape[0], *input_shape)

In [33]:
dataset    = tf.data.Dataset.from_tensor_slices((x_train, y_train))

dataset = dataset.batch(batch_size)

In [35]:
for img, label in dataset.take(1):
    print(img.shape)
    print(label.numpy())

(12, 28, 28, 1)
[5 0 4 1 9 2 1 3 1 4 3 5]


# Image Generator - Images in FolderClasses

In [91]:
from glob import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SIZE = [128, 128]


# Holding the paths:
train_path = r'D:\Machine Learning\datasets\bloodcells\TRAIN'
valid_path = r'D:\Machine Learning\datasets\bloodcells\TEST'
test_path = r'D:\Machine Learning\datasets\bloodcells\TEST_SIMPLE'

# Get List of files
image_files = glob(train_path + '/*/*.jp*g')
valid_image_files = glob(valid_path + '/*/*.jp*g')
test_images = glob(test_path + '/*/*.jp*g')


folders = glob(train_path + '/*')
num_classes = len(folders) 

# Train Generator
train_datagen = ImageDataGenerator(rescale=1.0/255.,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')


train_generator = train_datagen.flow_from_directory(directory=train_path,
                                                   target_size=IMAGE_SIZE,
                                                   class_mode='categorical',
                                                   batch_size=batch_size
                                                   )


# Validation Generator
valid_datagen = ImageDataGenerator(rescale=1.0/255.)

valid_generator = valid_datagen.flow_from_directory(
    valid_path,
    target_size=IMAGE_SIZE,
    class_mode="categorical",
    batch_size=batch_size,
)

Found 9957 images belonging to 4 classes.
Found 2487 images belonging to 4 classes.


In [92]:
train_dataset = tf.data.Dataset.from_generator(
    lambda: train_generator,
    output_types = (tf.float32, tf.float32),
    output_shapes = ([None, IMAGE_SIZE[0], IMAGE_SIZE[1], 3], [None, num_classes]),
)

valid_dataset = tf.data.Dataset.from_generator(
    lambda: valid_generator,
    output_types = (tf.float32, tf.float32),
    output_shapes = ([None, IMAGE_SIZE[0], IMAGE_SIZE[1], 3], [None, num_classes]),
)

In [94]:
train_dataset

<FlatMapDataset shapes: ((None, 128, 128, 3), (None, 4)), types: (tf.float32, tf.float32)>

In [90]:
# The dataset must be batched at the generator point - Here it doesnt work
# train_dataset.batch(8)
# valid_dataset.batch(8)

<BatchDataset shapes: ((None, None, 128, 128, 3), (None, None, 4)), types: (tf.float32, tf.float32)>

In [93]:
for img, label in valid_dataset.take(1):
    print(img.shape)
    print(label.shape)

(12, 128, 128, 3)
(12, 4)


# Segmentation Input - Images and Masks in Folders

In [95]:
train_paht = r'D:\GATE\07_Projects\03_rooftop_segmentation\data\train'

# image files
image_files = glob(train_paht + '/images/*.png')
# mask files
mask_files = glob(train_paht + '/masks/*.png')

print('number of images: ', len(image_files))
print('number of masks: ', len(mask_files))

number of images:  443
number of masks:  443


In [96]:
def get_image_mask_pair(list_image_files, list_mask_files):
    assert(len(list_image_files) == len(list_mask_files))
    return sorted(list_image_files), sorted(list_mask_files)

In [97]:
train_input_files, train_gt_files = get_image_mask_pair(image_files, mask_files)

In [98]:
image_files = tf.constant([train_input_files[:]])
gt_files = tf.constant([train_gt_files[:]])

In [99]:
dataset_train = tf.data.Dataset.from_tensor_slices({'image': tf.squeeze(image_files), 
                                                   'label': tf.squeeze(gt_files)})
dataset_train

In [104]:
for img, label in dataset_train.take(1):
    print(img)
#     print(img.shape)
#     print(label.shape)

image


In [105]:
def parse_function(filenames, resize_to=[128, 128]):
    """
    Parse files into input/label image pair.
    :param filenames:   Dict containing the two files (filenames['image'], filenames['label'])
    :param resize_to:   H x W Dimensions to resize the image and label to
    :return:            Input tensor, Label tensor
    """
    img_filename, gt_filename = filenames['image'], filenames['label']

    # Reading the files and returning their content as bytes:
    image_string, gt_string = tf.io.read_file(img_filename), tf.io.read_file(gt_filename)
    # Decoding those into images
    image_decoded = tf.io.decode_jpeg(image_string, channels=3)
    gt_decoded = tf.io.decode_png(gt_string, channels=1)
    
    # Converting image to float:
    image = tf.image.convert_image_dtype(image_decoded, tf.float32)
    gt = tf.cast(gt_decoded, tf.int32)
                                  
    # Resizing:
    image = tf.image.resize(image, resize_to)
    gt = tf.image.resize(gt, resize_to, method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    
    return image, gt

In [108]:
import functools

# We tell the dataset how to parse into images:
"""Why use functools.partial?"""
parse_fn = functools.partial(parse_function, resize_to=IMAGE_SIZE)
dataset_train = dataset_train.map(parse_fn)
print(dataset_train)

<MapDataset shapes: ((128, 128, 3), (128, 128, 1)), types: (tf.float32, tf.int32)>


In [110]:
for img, label in dataset_train.take(1):
    print(img.shape)
    print(label.shape)

(128, 128, 3)
(128, 128, 1)
